In [17]:
import random
import math
import csv
def generate_synthetic_data(num_samples=100):
    data = []
    for _ in range(num_samples):
        hour = random.randint(1, 24)  
        temperature = random.uniform(15, 40)
        if 9 <= hour <= 18 and temperature > 30:
            energy_usage = 1  
        else:
            energy_usage = random.choice([0, 1])        
        data.append([hour, temperature, energy_usage]) 
    return data

def save_to_csv(data, filename="synthetic_smart_grid_data.csv"):
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Hour", "Temperature", "EnergyUsage"])  
        writer.writerows(data)

def load_from_csv(filename="synthetic_smart_grid_data.csv"):
    with open(filename, mode="r") as file:
        reader = csv.reader(file)
        next(reader)  
        return [[float(row[0]), float(row[1]), int(row[2])] for row in reader]

def normalize(data):
    min_val = min(data)
    max_val = max(data)
    return [(x - min_val) / (max_val - min_val) for x in data]

def sigmoid(z):
    return 1 / (1 + math.exp(-z))

def gradient_descent(X, y, weights, learning_rate, epochs):
    n = len(y)
    for epoch in range(epochs):
        predictions = [sigmoid(sum(x * w for x, w in zip(X[i], weights))) for i in range(n)]
        
        gradients = [0] * len(weights)
        for j in range(len(weights)):
            gradients[j] = (1 / n) * sum((predictions[i] - y[i]) * X[i][j] for i in range(n))
        
        for j in range(len(weights)):
            weights[j] -= learning_rate * gradients[j]
        
        if epoch % 100 == 0:
            loss = -sum(y[i] * math.log(predictions[i]) + (1 - y[i]) * math.log(1 - predictions[i]) for i in range(n)) / n
            print(f"Epoch {epoch}, Loss: {loss}")
    
    return weights

def predict(X, weights, threshold=0.5):
    X_normalized = [[1] + [(val - min(col)) / (max(col) - min(col)) for val, col in zip(row, zip(*X))] for row in X]
    probabilities = [sigmoid(sum(x * w for x, w in zip(row, weights))) for row in X_normalized]
    return [1 if p >= threshold else 0 for p in probabilities]

if __name__ == "__main__":
    synthetic_data = generate_synthetic_data(100)
    save_to_csv(synthetic_data)
    data = load_from_csv()
    X = [row[:-1] for row in data]  
    y = [row[-1] for row in data] 
    X_normalized = [[(val - min(col)) / (max(col) - min(col)) for val, col in zip(row, zip(*X))] for row in X]
    X_normalized = [[1] + row for row in X_normalized]
    weights = [0] * len(X_normalized[0]) 
    learning_rate = 0.01
    epochs = 1000
    weights = gradient_descent(X_normalized, y, weights, learning_rate, epochs)
    test_data = [[6, 22], [15, 35], [20, 25]] 
    predictions = predict(test_data, weights)
    print("Predictions:", predictions)


Epoch 0, Loss: 0.6931471805599458
Epoch 100, Loss: 0.6707626375413259
Epoch 200, Loss: 0.6600224592738931
Epoch 300, Loss: 0.6545450282760763
Epoch 400, Loss: 0.6514820493342762
Epoch 500, Loss: 0.6495493593906739
Epoch 600, Loss: 0.6481590248771432
Epoch 700, Loss: 0.6470378768600499
Epoch 800, Loss: 0.6460573538637763
Epoch 900, Loss: 0.6451562404438131
Predictions: [1, 1, 1]
